In [49]:
import numpy as np
import pandas as pd


In [15]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

# MySQL database connection function
def connect_to_database():
    try:
        # Establishing connection to the database
        connection = mysql.connector.connect(
            host='crypto-matter.c5eq66ogk1mf.eu-central-1.rds.amazonaws.com',
            database='Crypto',
            user='Jing',  # Replace with your actual first name
            password='Crypto12!'
        )

        if connection.is_connected():
            db_info = connection.get_server_info()
            print("Connected to MySQL database, MySQL Server version: ", db_info)
            return connection

    except Error as e:
        print("Error while connecting to MySQL", e)
        return None



# Function to query merged data from crypto_lending_borrowing and crypto_price tables
def query_merged_crypto_data(connection):
    query = """
    SELECT clb.*, cp.*
    FROM crypto_lending_borrowing clb
    JOIN crypto_price cp 
        ON clb.crypto_symbol = cp.crypto_symbol
        AND clb.date = cp.date
    WHERE UPPER(clb.crypto_symbol) IN ('1INCHUSDT', 'BALUSDT', 'BATUSDT', 'CRVUSDT', 'ENJUSDT', 'ENSUSDT', 'KNCUSDT', 'LINKUSDT', 'MANAUSDT', 'MKRUSDT', 'RENUSDT', 'SNXUSDT', 'UNIUSDT', 'WBTCUSDT', 'YFIUSDT', 'ZRXUSDT')
    """
    cursor = connection.cursor()

    try:
        # Execute the query
        cursor.execute(query)

        # Fetch all results
        results = cursor.fetchall()

        # Get column names from cursor description
        columns = [desc[0] for desc in cursor.description]

        # Convert results to a Pandas DataFrame
        df = pd.DataFrame(results, columns=columns)

        return df

    except Error as e:
        print(f"Error: {e}")
        return None
    finally:
        cursor.close()

# Function to close the database connection
def query_quit(connection):
    if connection.is_connected():
        connection.close()
        print("MySQL connection is closed")


connection = connect_to_database()

if connection:
    # Query merged data
    merged_df = query_merged_crypto_data(connection)

    if merged_df is not None and not merged_df.empty:
        # Display first few rows of the DataFrame
        print("\nMerged DataFrame:")
        print(merged_df.head())

        # Save DataFrame to CSV
        merged_df.to_csv('merged_crypto_data.csv', index=False)
        print("\nMerged data saved to 'merged_crypto_data.csv'")
    else:
        print("\nNo data found after merging.")

    # Close the connection
    query_quit(connection)

Connected to MySQL database, MySQL Server version:  8.0.39

Merged DataFrame:
   id crypto_symbol                date lending_rate borrowing_rate  \
0   1       BATUSDT 2020-12-02 14:00:00   -50.000000     -50.000000   
1   2       BATUSDT 2020-12-02 15:00:00   -50.000000     -50.000000   
2   3       BATUSDT 2020-12-02 16:00:00   -50.000000     -50.000000   
3   4       BATUSDT 2020-12-02 17:00:00   -50.000000     -50.000000   
4   5       BATUSDT 2020-12-02 18:00:00   -50.000000     -50.000000   

  utilization_rate stable_borrow_rate crypto_symbol                date  high  \
0       -50.000000           0.030000       BATUSDT 2020-12-02 14:00:00  0.24   
1       -50.000000           0.030000       BATUSDT 2020-12-02 15:00:00  0.24   
2         0.013598           0.030000       BATUSDT 2020-12-02 16:00:00  0.24   
3         0.014834           0.030000       BATUSDT 2020-12-02 17:00:00  0.24   
4         0.014834           0.030000       BATUSDT 2020-12-02 18:00:00  0.25   

    low 

In [16]:
print(len(merged_df))
merged_df.head(3)

487675


,id,crypto_symbol,date,lending_rate,borrowing_rate,utilization_rate,stable_borrow_rate,crypto_symbol,date,high,low,close,adj_close,volume,market_cap,coin_supply
0,1,BATUSDT,2020-12-02 14:00:00,-50.000000,-50.000000,-50.000000,0.030000,BATUSDT,2020-12-02 14:00:00,0.24,0.24,0.24,0.24,1294545,None,None
1,2,BATUSDT,2020-12-02 15:00:00,-50.000000,-50.000000,-50.000000,0.030000,BATUSDT,2020-12-02 15:00:00,0.24,0.24,0.24,0.24,936344,None,None
2,3,BATUSDT,2020-12-02 16:00:00,-50.000000,-50.000000,0.013598,0.030000,BATUSDT,2020-12-02 16:00:00,0.24,0.24,0.24,0.24,724626,None,None


### Inputs: past 48 close (or return), volume(or growth rate), utilization rate, liquidity rate, variable borrow rate
### Output: next 24 liquidity rates, variable borrow rates

In [18]:
merged_df.loc[:, ~merged_df.columns.duplicated()]

,id,crypto_symbol,date,lending_rate,borrowing_rate,utilization_rate,stable_borrow_rate,high,low,close,adj_close,volume,market_cap,coin_supply
0,1,BATUSDT,2020-12-02 14:00:00,-50.000000,-50.000000,-50.000000,0.030000,0.24,0.24,0.24,0.24,1294545,None,None
1,2,BATUSDT,2020-12-02 15:00:00,-50.000000,-50.000000,-50.000000,0.030000,0.24,0.24,0.24,0.24,936344,None,None
2,3,BATUSDT,2020-12-02 16:00:00,-50.000000,-50.000000,0.013598,0.030000,0.24,0.24,0.24,0.24,724626,None,None
3,4,BATUSDT,2020-12-02 17:00:00,-50.000000,-50.000000,0.014834,0.030000,0.24,0.24,0.24,0.24,710071,None,None
4,5,BATUSDT,2020-12-02 18:00:00,-50.000000,-50.000000,0.014834,0.030000,0.25,0.24,0.24,0.24,1124485,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487670,515210,1INCHUSDT,2024-09-14 01:00:00,0.000001,0.659013,4.900089,-50.000000,0.27,0.27,0.27,0.27,369599,None,None
487671,515211,1INCHUSDT,2024-09-14 02:00:00,0.000001,0.659013,4.900089,-50.000000,0.27,0.27,0.27,0.27,280292,None,None
487672,515212,1INCHUSDT,2024-09-14 03:00:00,0.000001,0.659013,4.900089,-50.000000,0.27,0.27,0.27,0.27,320298,None,None
487673,515213,1INCHUSDT,2024-09-14 04:00:00,0.000001,0.659013,4.900089,-50.000000,0.27,0.27,0.27,0.27,208802,None,None


In [17]:
merged_df.columns.duplicated()

array([False, False, False, False, False, False, False,  True,  True,
       False, False, False, False, False, False, False])

In [19]:
# Drop the second occurrence of a specific column
merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]

In [20]:
print(len(merged_df[merged_df['borrowing_rate'] != -50])/len(merged_df))
print(len(merged_df[merged_df['lending_rate'] != -50])/len(merged_df))
print(len(merged_df[merged_df['utilization_rate'] != -50])/len(merged_df))

0.858625108935254
0.8607925360127134
0.9998298046854975


In [21]:
# data without NA rows if we needed 
filtered_df = merged_df[(merged_df['borrowing_rate'] != -50)&(merged_df['lending_rate'] != -50)&(merged_df['utilization_rate'] != -50)]
filtered_df.reset_index(inplace=True, drop=True)
print(len(filtered_df)/ len(merged_df))

0.8584610652586251


In [37]:
test_df = filtered_df[filtered_df['crypto_symbol'] == 'BATUSDT']
test_df.reset_index(inplace=True, drop=True)
print(len(test_df))
test_df.head(3)

28159


,id,crypto_symbol,date,lending_rate,borrowing_rate,utilization_rate,stable_borrow_rate,high,low,close,adj_close,volume,market_cap,coin_supply
0,22,BATUSDT,2020-12-03 11:00:00,0.000344,0.029157,0.012362,0.031099,0.25,0.24,0.25,0.25,523778,None,None
1,23,BATUSDT,2020-12-03 12:00:00,0.000179,0.015617,0.007912,0.032033,0.25,0.24,0.24,0.24,658023,None,None
2,24,BATUSDT,2020-12-03 13:00:00,0.000008,0.001246,0.008011,0.031780,0.25,0.24,0.24,0.24,661386,None,None


In [38]:
test_df.sort_values('date', ascending=False).head(10)

,id,crypto_symbol,date,lending_rate,borrowing_rate,utilization_rate,stable_borrow_rate,high,low,close,adj_close,volume,market_cap,coin_supply
28158,33158,BATUSDT,2024-09-14 03:00:00,0.000000,0.200054,-0.160560,0.034384,0.17,0.17,0.17,0.17,175363,None,None
28157,33157,BATUSDT,2024-09-14 02:00:00,0.000000,0.200054,-0.160560,0.034384,0.17,0.17,0.17,0.17,139283,None,None
28156,33156,BATUSDT,2024-09-14 01:00:00,0.000000,0.200054,-0.160560,0.034384,0.18,0.17,0.17,0.17,109077,None,None
28155,33155,BATUSDT,2024-09-14 00:00:00,0.000000,0.200054,-0.160560,0.034384,0.18,0.17,0.17,0.17,101143,None,None
28154,33154,BATUSDT,2024-09-13 23:00:00,0.000000,0.200054,-0.160560,0.034384,0.18,0.17,0.18,0.18,59155,None,None
28153,33153,BATUSDT,2024-09-13 22:00:00,0.000000,0.200054,-0.160560,0.034384,0.18,0.18,0.18,0.18,108665,None,None
28152,33152,BATUSDT,2024-09-13 21:00:00,0.000000,0.200054,-0.160560,0.034384,0.18,0.17,0.18,0.18,155450,None,None
28151,33151,BATUSDT,2024-09-13 20:00:00,0.000000,0.200054,-0.160560,0.034384,0.18,0.17,0.17,0.17,177871,None,None
28150,33150,BATUSDT,2024-09-13 19:00:00,0.000000,0.200054,-0.160560,0.034384,0.18,0.17,0.17,0.17,126856,None,None
28149,33149,BATUSDT,2024-09-13 18:00:00,0.000000,0.200054,-0.160560,0.034384,0.18,0.17,0.18,0.18,160280,None,None


In [39]:
test_df.sort_values('date', ascending=False).iloc[-40:-30]

,id,crypto_symbol,date,lending_rate,borrowing_rate,utilization_rate,stable_borrow_rate,high,low,close,adj_close,volume,market_cap,coin_supply
39,61,BATUSDT,2020-12-05 02:00:00,0.000000,0.000028,0.000135,0.030631,0.23,0.23,0.23,0.23,358612,None,None
38,60,BATUSDT,2020-12-05 01:00:00,0.000000,0.000028,0.000135,0.030631,0.23,0.23,0.23,0.23,1540375,None,None
37,59,BATUSDT,2020-12-05 00:00:00,0.000000,0.000028,0.000135,0.030631,0.23,0.23,0.23,0.23,716057,None,None
36,58,BATUSDT,2020-12-04 23:00:00,0.000000,0.000028,0.000135,0.030631,0.23,0.23,0.23,0.23,943380,None,None
35,57,BATUSDT,2020-12-04 22:00:00,0.000000,0.000028,0.000135,0.030631,0.23,0.23,0.23,0.23,1926443,None,None
34,56,BATUSDT,2020-12-04 21:00:00,0.000000,0.000029,0.000361,0.030581,0.24,0.23,0.23,0.23,628793,None,None
33,55,BATUSDT,2020-12-04 20:00:00,0.000000,0.000019,0.001713,0.030030,0.24,0.24,0.24,0.24,241575,None,None
32,54,BATUSDT,2020-12-04 19:00:00,0.000000,0.000002,0.000090,0.030020,0.24,0.24,0.24,0.24,318694,None,None
31,53,BATUSDT,2020-12-04 18:00:00,0.000000,0.000004,0.000027,0.030006,0.24,0.24,0.24,0.24,233578,None,None
30,52,BATUSDT,2020-12-04 17:00:00,0.000000,0.000004,0.000027,0.030006,0.24,0.24,0.24,0.24,305424,None,None


In [40]:
test_df['date'] = pd.to_datetime(test_df['date'])


C:\Users\hgnik\AppData\Local\Temp\ipykernel_40792\2818006634.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['date'] = pd.to_datetime(test_df['date'])


In [41]:
test_df['year'] = test_df['date'].dt.year
test_df['year'].value_counts(dropna=False)

C:\Users\hgnik\AppData\Local\Temp\ipykernel_40792\3133557596.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['year'] = test_df['date'].dt.year


year
2021    7681
2022    7247
2023    7151
2024    5489
2020     591
Name: count, dtype: int64

In [42]:
test_df.groupby('year')['borrowing_rate'].mean()

year
2020    0.059911
2021    0.026441
2022    0.026179
2023     0.03438
2024    0.224616
Name: borrowing_rate, dtype: object

In [44]:
test_df['borrowing_rate'].mean()

0.0677223126531482

In [43]:
len(test_df[(test_df['year'] == 2020) & (test_df['borrowing_rate'] < 0.01)])

78

In [53]:
# Define a function to calculate outlier bounds using IQR
def calculate_iqr_bounds(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = max(Q1 - 1.5 * IQR, 0)
    upper_bound = Q3 + 1.5 * IQR
    return lower_bound, upper_bound

print(calculate_iqr_bounds(test_df['borrowing_rate'].astype(float)))



(0, 0.111785)


In [70]:
test_df = test_df[['crypto_symbol', 'date', 'lending_rate',	'borrowing_rate','utilization_rate','close', 'volume']]
test_df[['lending_rate',	'borrowing_rate','utilization_rate','close', 'volume']] = test_df[['lending_rate','borrowing_rate','utilization_rate','close', 'volume']].astype(float)

In [71]:
# Calculate intervals for each column
intervals = {}
for column in ['lending_rate',	'borrowing_rate','utilization_rate']:
    lower, upper = calculate_iqr_bounds(test_df[column])
    intervals[column] = {'lower_bound': lower, 'upper_bound': upper}

# Display intervals
for col, bounds in intervals.items():
    print(f"{col.capitalize()}: Min = {bounds['lower_bound']:.4f}, Max = {bounds['upper_bound']:.4f}")

Lending_rate: Min = 0.0000, Max = 0.0100
Borrowing_rate: Min = 0.0000, Max = 0.1118
Utilization_rate: Min = 0.0000, Max = 0.2839


In [80]:
# First Loop: Calculate intervals for each column without modifying the DataFrame
intervals = {}
for column in ['lending_rate', 'borrowing_rate', 'utilization_rate']:
    lower, upper = calculate_iqr_bounds(test_df[column].astype(float))
    intervals[column] = {'lower_bound': lower, 'upper_bound': upper}

# Second Loop: Filter rows based on the pre-calculated intervals
reduced_df = test_df.copy()
for column in ['lending_rate', 'borrowing_rate', 'utilization_rate']:
    lower_bound = intervals[column]['lower_bound']
    upper_bound = intervals[column]['upper_bound']
    # Apply filtering based on pre-calculated bounds
    reduced_df = reduced_df[(reduced_df[column] > lower_bound) & (reduced_df[column] < upper_bound)]

# Display the filtered DataFrame
print(len(reduced_df))

20469


In [81]:
reduced_df['year'] = reduced_df['date'].dt.year

In [82]:
reduced_df.groupby('year')['borrowing_rate'].mean()

year
2020    0.025136
2021    0.021610
2022    0.018879
2023    0.023183
2024    0.020504
Name: borrowing_rate, dtype: float64

### Now it makes Sense!

In [93]:
def calculate_hourly_returns(df, date_col, close_col):
    """
    Calculates returns based on the close price, only if the date difference is 1 hour.

    Args:
        df (pd.DataFrame): The DataFrame containing the time series data.
        date_col (str): The name of the datetime column.
        close_col (str): The name of the close price column.

    Returns:
        pd.Series: A Series containing the calculated returns or None for invalid rows.
    """
    # Ensure the date column is in datetime format
    df[date_col] = pd.to_datetime(df[date_col])
    
    # Sort by date to ensure sequential order
    df = df.sort_values(by=date_col).reset_index(drop=True)
    
    # Calculate the time difference between consecutive rows in hours
    time_diff = df[date_col].diff().dt.total_seconds() / 3600
    
    # Calculate returns only for rows where time_diff == 1 hour
    returns = np.where(
        time_diff == 1,
        (df[close_col] - df[close_col].shift(1)) / df[close_col].shift(1),
        None
    )
    
    return pd.Series(returns, index=df.index)

In [103]:
reduced_df['returns'] = calculate_hourly_returns(reduced_df, 'date', 'close')
reduced_df = reduced_df[reduced_df['returns'].notna()]
reduced_df.reset_index(inplace=True, drop=True)
reduced_df['returns'].notna().sum()

18340

In [111]:
print(len(reduced_df))

18340


In [113]:
reduced_df[['date','returns','close']]

,date,returns,close
0,2020-12-03 12:00:00,-0.04,0.24
1,2020-12-03 13:00:00,0.0,0.24
2,2020-12-03 14:00:00,0.041667,0.25
3,2020-12-03 15:00:00,0.0,0.25
4,2020-12-03 16:00:00,-0.04,0.24
...,...,...,...
18335,2023-09-14 19:00:00,0.0,0.17
18336,2023-09-14 20:00:00,0.0,0.17
18337,2023-09-14 21:00:00,0.0,0.17
18338,2023-09-14 22:00:00,0.0,0.17


In [115]:
def count_valid_windows(df, date_col, window_size):
    """
    Counts the number of valid windows of a given size where each row in the window
    has a `date_col` that is exactly 1 hour after the previous row.

    Args:
        df (pd.DataFrame): The DataFrame containing the time series data.
        date_col (str): The name of the datetime column.
        window_size (int): The size of the valid window to check for.

    Returns:
        int: The number of valid windows found.
    """
    # Ensure the date column is in pandas datetime format
    df[date_col] = pd.to_datetime(df[date_col])
    
    # Sort by the datetime column to ensure chronological order
    df = df.sort_values(by=date_col).reset_index(drop=True)
    
    # Initialize the count of valid windows
    valid_window_count = 0

    # Iterate through the DataFrame to check for valid windows
    for i in range(len(df) - window_size + 1):
        # Extract the potential window
        window = df.iloc[i:i + window_size]
        
        # Check if all rows in the window have exactly 1-hour difference
        time_diffs = window[date_col].diff().dt.total_seconds()[1:]
        if (time_diffs == 3600).all():
            valid_window_count += 1

    return valid_window_count

print(count_valid_windows(reduced_df, 'date', 1))

18340


In [104]:
reduced_df.columns

Index(['crypto_symbol', 'date', 'lending_rate', 'borrowing_rate',
       'utilization_rate', 'close', 'volume', 'year', 'returns'],
      dtype='object')

In [105]:
reduced_df.isna().sum()

crypto_symbol       0
date                0
lending_rate        0
borrowing_rate      0
utilization_rate    0
close               0
volume              0
year                0
returns             0
dtype: int64

In [106]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# MinMax scaling
scaler = MinMaxScaler(feature_range=(0, 1))  # Default range is (0, 1)

scaled_df = reduced_df.copy()
scaled_df['lending_rate'] = scaler.fit_transform(reduced_df[['lending_rate']])
scaled_df['borrowing_rate'] = scaler.fit_transform(reduced_df[['borrowing_rate']])
scaled_df['utilization_rate'] = scaler.fit_transform(reduced_df[['utilization_rate']])
scaled_df['close'] = scaler.fit_transform(reduced_df[['close']])
scaled_df['volume'] = scaler.fit_transform(reduced_df[['volume']])
scaled_df['returns'] = scaler.fit_transform(reduced_df[['returns']])



In [107]:
# now we have a dataframe that does not have any NA and ay outlier, but its time series is corrupted, therefore we need valid windows
def extract_valid_windows(df, date_col, input_window, target_window, input_columns, target_columns):
    """
    Extracts valid windows from a time series DataFrame for LSTM training.
    
    Args:
        df (pd.DataFrame): The time series DataFrame with a datetime column.
        date_col (str): The name of the datetime column.
        input_window (int): The number of timesteps for the input sequence.
        target_window (int): The number of timesteps for the target sequence.
        input_columns (list of str): List of column names to include in the input data.
        target_columns (list of str): List of column names to include in the target data.
        
    Returns:
        inputs (list of np.ndarray): List of valid input sequences.
        targets (list of np.ndarray): List of corresponding target sequences.
    """
    # Sort by the datetime column to ensure the time series is ordered
    df = df.sort_values(by=date_col).reset_index(drop=True)
    
    # Ensure the datetime column is in pandas datetime format
    df[date_col] = pd.to_datetime(df[date_col])
    
    # Identify valid consecutive rows (1-hour apart)
    time_diffs = df[date_col].diff().dt.total_seconds()
    valid_indices = time_diffs == 3600  # 1 hour = 3600 seconds
    
    # Mark valid sequences
    valid_sequence_flags = valid_indices | valid_indices.shift(-1, fill_value=False)
    df = df[valid_sequence_flags].reset_index(drop=True)

    # Prepare inputs and targets
    inputs, targets = [], []
    total_window = input_window + target_window

    for i in range(len(df) - total_window + 1):
        # Extract a potential window of size `total_window`
        window = df.iloc[i:i+total_window]
        
        # Check if all rows in the window are 1-hour apart
        if (window[date_col].diff().dt.total_seconds()[1:] == 3600).all():
            # Split into input and target based on specified columns
            input_data = window.iloc[:input_window][input_columns].values
            target_data = window.iloc[input_window:][target_columns].values
            inputs.append(input_data)
            targets.append(target_data)

    return np.array(inputs), np.array(targets)

In [108]:

inputs, targets = extract_valid_windows(scaled_df,'date', 40, 10, ['lending_rate',	'borrowing_rate','utilization_rate','returns', 'volume'], ['lending_rate','borrowing_rate'])


In [109]:
print(inputs.shape)
print(targets.shape)

(7170, 40, 5)
(7170, 10, 2)


In [110]:
inputs[0]

array([[0.23524681, 0.20467033, 0.48333161, 0.59017595, 0.00305671],
       [0.23524681, 0.20467033, 0.48333161, 0.59017595, 0.00286493],
       [0.23524681, 0.20467033, 0.48333161, 0.67844575, 0.00214928],
       [0.23524681, 0.20467033, 0.48333161, 0.59017595, 0.00616256],
       [0.23524681, 0.20467033, 0.48333161, 0.59017595, 0.00567901],
       [0.23524681, 0.20467033, 0.48333161, 0.50530115, 0.00842222],
       [0.23524681, 0.20467033, 0.48333161, 0.59017595, 0.00472835],
       [0.23524681, 0.20467033, 0.48333161, 0.59017595, 0.00769585],
       [0.23524681, 0.20467033, 0.48333161, 0.59017595, 0.00746698],
       [0.23524681, 0.20467033, 0.48333161, 0.59017595, 0.0096813 ],
       [0.23524681, 0.20467033, 0.48333161, 0.59017595, 0.00510086],
       [0.23514628, 0.20464151, 0.483239  , 0.59017595, 0.00531153],
       [0.23494521, 0.20452624, 0.48296473, 0.59017595, 0.00604609],
       [0.23494521, 0.20452624, 0.48296473, 0.59017595, 0.0169468 ],
       [0.23494521, 0.20452624, 0.